In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, explode, split, desc

In [11]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Movie Ratings Analysis") \
    .getOrCreate()

In [12]:
# Load the Movies CSV file
movies_file_path = "/Users/sampreethshetty/Downloads/ml-latest-small/movies.csv"  # Replace with your movies file path
movies_df = spark.read.csv(movies_file_path, header=True, inferSchema=True)

# Load the Ratings CSV file
ratings_file_path = "/Users/sampreethshetty/Downloads/ml-latest-small/ratings.csv"  # Replace with your ratings file path
ratings_df = spark.read.csv(ratings_file_path, header=True, inferSchema=True)

# Create Temporary Views
movies_df.createOrReplaceTempView("Movies")
ratings_df.createOrReplaceTempView("Ratings")


In [13]:
# SQL Query to Find Top 10 Highest-Rated Movies with at Least 10 Ratings
query = """
    SELECT 
        m.MovieID,
        m.Title,
        ROUND(AVG(r.Rating), 2) AS Avg_Rating,
        COUNT(r.Rating) AS Total_Ratings
    FROM 
        Movies m
    JOIN 
        Ratings r
    ON 
        m.MovieID = r.MovieID
    GROUP BY 
        m.MovieID, m.Title
    HAVING 
        COUNT(r.Rating) >= 10
    ORDER BY 
        Avg_Rating DESC
    LIMIT 10
"""

In [14]:
top_rated_movies = spark.sql(query)

# Show the result
print("Top 10 Highest-Rated Movies with at Least 10 Ratings:")
top_rated_movies.show()


Top 10 Highest-Rated Movies with at Least 10 Ratings:
+-------+--------------------+----------+-------------+
|MovieID|               Title|Avg_Rating|Total_Ratings|
+-------+--------------------+----------+-------------+
|   1041|Secrets & Lies (1...|      4.59|           11|
|   3451|Guess Who's Comin...|      4.55|           11|
|   1178|Paths of Glory (1...|      4.54|           12|
|   1104|Streetcar Named D...|      4.48|           20|
|   2360|Celebration, The ...|      4.46|           12|
|    318|Shawshank Redempt...|      4.43|          317|
|   1217|          Ran (1985)|      4.43|           15|
|    951|His Girl Friday (...|      4.39|           14|
|   1927|All Quiet on the ...|      4.35|           10|
|   3468| Hustler, The (1961)|      4.33|           18|
+-------+--------------------+----------+-------------+



In [15]:
# Show loaded data
print("Movies Data:")
movies_df.show(5)

Movies Data:
+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [16]:
print("Ratings Data:")
ratings_df.show(5)

Ratings Data:
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [24]:
# 1. Find the most active users (users who have rated the most movies)
most_active_users = ratings_df.groupBy("UserID") \
    .agg(count("Rating").alias("Total_Ratings")) \
    .orderBy(desc("Total_Ratings"))

print("Most Active Users (Users who rated the most movies):")
most_active_users.show(5)

Most Active Users (Users who rated the most movies):
+------+-------------+
|UserID|Total_Ratings|
+------+-------------+
|   414|         2698|
|   599|         2478|
|   474|         2108|
|   448|         1864|
|   274|         1346|
+------+-------------+
only showing top 5 rows



In [20]:
# 2. Sort the movies by name in alphabetical order
sorted_movies = movies_df.orderBy("Title")

print("Movies sorted alphabetically:")
sorted_movies.select("Title").show(10, truncate=False)

Movies sorted alphabetically:
+---------------------------------------+
|Title                                  |
+---------------------------------------+
|"11'09""01 - September 11 (2002)"      |
|'71 (2014)                             |
|'Hellboy': The Seeds of Creation (2004)|
|'Round Midnight (1986)                 |
|'Salem's Lot (2004)                    |
|'Til There Was You (1997)              |
|'Tis the Season for Love (2015)        |
|'burbs, The (1989)                     |
|'night Mother (1986)                   |
|(500) Days of Summer (2009)            |
+---------------------------------------+
only showing top 10 rows



In [21]:
# 3. Calculate the average rating per genre
# Assume the "Genres" column in the movies_df contains genres separated by "|"
movies_with_genres = movies_df.withColumn("Genre", explode(split(col("Genres"), "\\|")))

# Join movies with ratings on MovieID
movies_ratings_joined = movies_with_genres.join(ratings_df, on="MovieID")

# Calculate average rating per genre
avg_rating_per_genre = movies_ratings_joined.groupBy("Genre") \
    .agg(avg("Rating").alias("Avg_Rating")) \
    .orderBy(desc("Avg_Rating"))

print("Average Rating per Genre:")
avg_rating_per_genre.show()

Average Rating per Genre:
+------------------+------------------+
|             Genre|        Avg_Rating|
+------------------+------------------+
|         Film-Noir| 3.920114942528736|
|               War|   3.8082938876312|
|       Documentary| 3.797785069729286|
|             Crime| 3.658293867274144|
|             Drama|3.6561844113718758|
|           Mystery| 3.632460255407871|
|         Animation|3.6299370349170004|
|              IMAX| 3.618335343787696|
|           Western| 3.583937823834197|
|           Musical|3.5636781053649105|
|         Adventure|3.5086089151939075|
|           Romance|3.5065107040388437|
|          Thriller|3.4937055799183425|
|           Fantasy|3.4910005070136894|
|(no genres listed)|3.4893617021276597|
|            Sci-Fi| 3.455721162210752|
|            Action| 3.447984331646809|
|          Children| 3.412956125108601|
|            Comedy|3.3847207640898267|
|            Horror| 3.258195034974626|
+------------------+------------------+



In [25]:
movies_with_genres.show(5)

+-------+----------------+--------------------+---------+
|movieId|           title|              genres|    Genre|
+-------+----------------+--------------------+---------+
|      1|Toy Story (1995)|Adventure|Animati...|Adventure|
|      1|Toy Story (1995)|Adventure|Animati...|Animation|
|      1|Toy Story (1995)|Adventure|Animati...| Children|
|      1|Toy Story (1995)|Adventure|Animati...|   Comedy|
|      1|Toy Story (1995)|Adventure|Animati...|  Fantasy|
+-------+----------------+--------------------+---------+
only showing top 5 rows



In [7]:
# Stop Spark Session
spark.stop()